In [1]:
from pathlib import Path

import matplotlib.pyplot as plt

import h5py
import numpy as np
from magtense.utils import plot_M_thin_film

from koopmag.database import create_db_mp
from koopmag.utils import plot_dynamics

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset

from mpl_toolkits.axes_grid1 import make_axes_locatable

def add_colorbar(im, fig, ax) -> None:
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im, cax=cax, orientation='vertical')

In [3]:
datapath = Path().cwd().parent / "data"
db = h5py.File(datapath / "20_200_36_9.h5", "r")

print("Attributes of database:")
for key in db.attrs:
    print(f"{key}: {db.attrs[key]}")

print("\nData in the database:")
for key in db:
    print(f"{key}: {db[key].shape}")

Attributes of database:
grid_size: [5.00e-07 1.25e-07 3.00e-09]
h_ext_angle: [  0 360]
h_ext_norm: [ 0 50]
res: [36  9  1]
seed: 0
t_per_step: 1e-11
t_steps: 200

Data in the database:
field: (20, 3)
sequence: (20, 200, 36, 9, 3)


In [36]:
Hs = np.array(db["field"])

data = np.array(db["sequence"])

n_seq, n_tsteps, data_dim = *data.shape[:2], data.shape[2:]

data = data.reshape((n_seq, n_tsteps, -1))
Xs = data[:, :-1, :]
Ys = data[:,  1:, :]

print(f"Concatenated dimension: from {data_dim} to ({np.prod(data_dim)},)")

print(f"Xs shape: {Xs.shape} (n_seq, n_tsteps, n_x * n_y * 3)")
print(f"Ys shape: {Xs.shape} (n_seq, n_tsteps, n_x * n_y * 3)")

Concatenated dimension: from (36, 9, 3) to (972,)
Xs shape: (20, 199, 972) (n_seq, n_tsteps, n_x * n_y * 3)
Ys shape: (20, 199, 972) (n_seq, n_tsteps, n_x * n_y * 3)


In [38]:
Xtorch = torch.tensor(Xs, dtype=torch.float32)
Ytorch = torch.tensor(Ys, dtype=torch.float32)

Htorch = torch.stack(
    [
        torch.stack(
            [torch.tensor(Hs[i], dtype=torch.float32) for _ in range(n_tsteps - 1)]
        )
        for i in range(n_seq)
    ]
)

Htorch.shape

torch.Size([20, 199, 3])

### Koopman Autoencoder

In [27]:
class FCEncoder(nn.Module):
    '''
    Encoder block for Autoencoder with fully connected layers.
    takes as input a list of number of nodes per layer
    and an activation function
    '''
    def __init__(self, dim_list, act_fn=nn.Tanh) -> None:
        super().__init__()

        module_list = []
        for i in range(len(dim_list)-1):
            module_list.append(nn.Linear(dim_list[i], dim_list[i+1]))
            if i != len(dim_list)-2:
                module_list.append(act_fn())

        self.fc = nn.Sequential(*module_list)

    def forward(self, x) -> torch.Tensor:
        return self.fc(x)


class FCDecoder(nn.Module):
    '''
    Decoder block for Autoencoder with fully connected layers.
    takes as input a list of number of nodes per layer
    and an activation function
    '''
    def __init__(self, dim_list, act_fn=nn.Tanh) -> None:
        super().__init__()
        module_list = []
        for i in range(len(dim_list)-1, 0, -1):
            module_list.append(nn.Linear(dim_list[i], dim_list[i-1]))
            if i != 1:
                module_list.append(act_fn())

        self.fc = nn.Sequential(*module_list)

    def forward(self, x) -> torch.Tensor:
        return self.fc(x)


class DeepKoopman(nn.Module):

    def __init__(self, dim_list, act_fn=nn.Tanh) -> None:
        super().__init__()
        self.encoder = FCEncoder(dim_list, act_fn)
        self.decoder = FCDecoder(dim_list, act_fn)

        self.B = nn.Parameter(torch.empty(3, dim_list[-1])) # shape (n_magnetization, d)
        nn.init.xavier_uniform_(self.B, gain=0.01)          # initialise B matrix values

    def compute_linear_operator(self, Xtilde, Ytilde) -> torch.Tensor:

        self.A = torch.linalg.pinv(Xtilde) @ Ytilde
        return self.A

    def forward(self, X, Y, U=None) -> tuple[torch.Tensor, torch.Tensor]:
        '''
        params:
        X: input data (n x m)
        Y: target data (n x m)
        U: external effect (n x 3)
        '''
        T = X.shape[0]
        T_half = T // 2
        Xtilde = self.encoder(X)    # encode X: (n x m)  -> Xtilde: (n x d)
        Ytilde = self.encoder(Y)    # encode Y: (n x m)  -> Ytilde: (n x d)

        ext_effect = U[:T_half, :] @ self.B if U is not None else 0 # shape (n//2 x d)

        # compute linear operator according to 
        # A = (Ytilde - B @ U) @ Xtilde^+
        A = self.compute_linear_operator(
            Xtilde[:T_half,:], Ytilde[:T_half,:] - ext_effect
            )

        Ytilde_pred_list = [Xtilde[0] @ A]
        for t in range(1, T):
            next_pred = Ytilde_pred_list[t-1] @ A
            if U is not None:
                next_pred += U[t,:] @ self.B
            Ytilde_pred_list.append(next_pred)
        
        Ytilde_pred = torch.stack(Ytilde_pred_list, dim=0)
        
        Xhat = self.decoder(Xtilde)
        Yhat = self.decoder(Ytilde_pred)

        return Xhat, Yhat
    
    
    def predict(self, X, T, U=None) -> torch.Tensor:
        if len(X.shape) == 1:
            X = X.unsqueeze(0)  # make it (1, input_dim)

        with torch.no_grad():
            Xtilde0 = self.encoder(X)    # encode X: (n x m)  -> Xtilde: (n x d)
            Ytilde_pred = torch.zeros((T, Xtilde0.shape[1])).to(Xtilde0.device)
            Ytilde_pred[0] = Xtilde0 @ self.A 
            if U is not None:
                Ytilde_pred[0] += U[0,:] @ self.B
            for t in range(1, T):
                Ytilde_pred[t] = Ytilde_pred[t-1] @ self.A
                if U is not None:
                    Ytilde_pred[t] += U[t,:] @ self.B
            
            Yhat = self.decoder(Ytilde_pred)
        return Yhat